In [1]:
from threading import Lock
from typing import Optional, Literal
from contextvars import ContextVar

import dspy
from dspy.utils.callback import BaseCallback
from pydantic import BaseModel, Field, field_validator

# config
file_lock = Lock()
class LoggingCallback(BaseCallback): # FIXME: this will influence the prompt format, idk.
    def on_module_end(self, call_id, outputs, exception = None):
        with file_lock:
            with open('log.txt', 'w', encoding='utf8') as file:
                for k, v in outputs.items():
                    file.write(f"{k}: {v}\n")

class SwappedChatAdapter(dspy.adapters.ChatAdapter):
    def format(self, signature, demos, inputs):
        messages:list = super().format(signature, demos, inputs)
        systeme_message = messages.pop(0)
        messages.insert(-1, systeme_message)
        return messages

lm = dspy.LM('openai/gpt-4o', cache=False)
dspy.configure(lm=lm, adapter=SwappedChatAdapter())
additional_args = ContextVar('additional_args', default={})

class Target(BaseModel):
    target_formula: str = Field(description='make sure it is a valid chemical formula')
    amount_var: dict[str, list[float]] = Field(description='the amount variable in the formula, e.g. AxBC, {x: [1, 2]}')
    element_var: dict[str, list[str]] = Field(description='the element variable in the formula')

class Reaction(BaseModel):
    precursors: list[str] = Field(description='ensure it is a valid chemical formula')
    additives: list[str]
    target: Target
    reaction_type: Literal['solid-state', 'sol-gel', 'co-precipitation', 'hydrothermal', 'flux', 'others']

    # @field_validator('precursors')
    # @classmethod
    # def validate_precusors(cls, v):
    #     if not v:
    #         raise ValueError('precursors should not be empty')
    #     return v
    

class QA(dspy.Signature):
    # """extract all chmemical reactions consitituent from the text.
    # Note: please only include those reactions in which precursors and target are explicitly mentioned in the text.
    # Intermediate reaction should be extracted as well."""
    """extract all chmemical reactions consitituent from the text.
    Note: please only include those reactions in which precursors and target are explicitly mentioned in the text.
    Intermediate reaction should be extracted as well."""
    text: str = dspy.InputField(desc='a piece of text which may contains chemical reactions')
    reactions: Optional[list[Reaction]] = dspy.OutputField(desc='the reactions extracted from the text, return null if no reaction found')

class ExtractReactionWithType(dspy.Module):
    def __init__(self):
        self.predictor = dspy.ChainOfThought(signature=QA)

    def forward(self, text):
        prediction = self.predictor(text=text)
        return prediction

e:\anaconda\envs\sisyphus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
compiled_program = ExtractReactionWithType()
compiled_program.load('compiled_direct_with_element_var_version_2')
uncompiled_program = ExtractReactionWithType()
from copy import deepcopy
vanilla_QA = deepcopy(QA)
vanilla_QA.__doc__ = "extract reactions from text"
vanilla_program = dspy.Predict(vanilla_QA)

In [3]:
vanilla_program.signature

QA(text -> reactions
    instructions='extract reactions from text'
    text = Field(annotation=str required=True json_schema_extra={'desc': 'a piece of text which may contains chemical reactions', '__dspy_field_type': 'input', 'prefix': 'Text:'})
    reactions = Field(annotation=Union[list[Reaction], NoneType] required=True json_schema_extra={'desc': 'the reactions extracted from the text, return null if no reaction found', '__dspy_field_type': 'output', 'prefix': 'Reactions:'})
)

In [9]:
vanilla_program.signature.instructions = """Your objective is to extract all chemical reactions described in the text, regardless of prominence. Each reaction should include:
1. Explicitly mentioned precursors and target products.
2. Reaction type, as classified into solid-state, sol-gel, hydrothermal, co-precipitation, flux, or others.

Ensure reasoning includes all reactions described.
"""

In [10]:
vanilla_program.signature.instructions

'Your objective is to extract all chemical reactions described in the text, regardless of prominence. Each reaction should include:\n1. Explicitly mentioned precursors and target products.\n2. Reaction type, as classified into solid-state, sol-gel, hydrothermal, co-precipitation, flux, or others.\n\nEnsure reasoning includes all reactions described.'

In [3]:
demos = compiled_program.predictor.demos
keeped_demos = []
keeped_demos.append(demos[0])
keeped_demos.append(demos[2])
keeped_demos.append(demos[3])
compiled_program.predictor._predict.demos = keeped_demos
compiled_program.predictor.demos

[{'augmented': True,
  'text': '(1−x)(Ca0.88Sr0.12)TiO3–x(Bi0.5Na0.5)TiO3 [hereafter referred to as (1−x)CST–xBNT] ceramics (x\u2009=\u20090.025, 0.050, 0.075, 0.10) were prepared by the conventional solid-state reaction method. The starting materials were high-purity grade powders (>\u200999%): CaCO3, SrCO3, Bi2O3, Na2CO3 and TiO2. CST and BNT powders were synthesized separately by mixing the starting materials according to the desired stoichiometry, and milled with ZrO2 balls and deionized water for 6 h in nylon jars. After drying, the powders were calcined at 1100 °C for 3 h and 850 °C for 3 h, respectively. The resulting powders were re-milled for 6 h. After that, the dried and re-milled mixtures were granulated with appropriate poly vinyl alcohol (PVA) as binder and uniaxial pressed at 300 MPa into cylinders with approximate size of 15 mm in diameter and 7.3 mm in thickness. The cylinders with x\u2009=\u20090.05, 0.075 and 0.1 were sintered in air at 1225–1300 °C for 3 h, and the 

In [4]:
compiled_program.predictor.signature

QA(text -> reactions
    instructions='extract all chmemical reactions consitituent from the text.\nNote: please only include those reactions in which precursors and target are explicitly mentioned in the text.\nIntermediate reaction should be extracted as well.'
    text = Field(annotation=str required=True json_schema_extra={'desc': 'a piece of text which may contains chemical reactions', '__dspy_field_type': 'input', 'prefix': 'Text:'})
    reactions = Field(annotation=Union[list[Reaction], NoneType] required=True json_schema_extra={'desc': 'the reactions extracted from the text, return null if no reaction found', '__dspy_field_type': 'output', 'prefix': 'Reactions:'})
)

In [5]:
compiled_program.predictor.extended_signature.instructions = """Your objective is to extract all chemical reactions described in the text, regardless of prominence. Each reaction should include:
1. Explicitly mentioned precursors and target products.
2. Reaction type, as classified into solid-state, sol-gel, hydrothermal, co-precipitation, flux, or others.

Ensure reasoning includes all reactions described, mentioning each reaction process briefly.

Examples are provided for reference."""
compiled_program.predictor.extended_signature

StringSignature(text -> reasoning, reactions
    instructions='Your objective is to extract all chemical reactions described in the text, regardless of prominence. Each reaction should include:\n1. Explicitly mentioned precursors and target products.\n2. Reaction type, as classified into solid-state, sol-gel, hydrothermal, co-precipitation, flux, or others.\n\nEnsure reasoning includes all reactions described, mentioning each reaction process briefly.\n\nExamples are provided for reference.'
    text = Field(annotation=str required=True json_schema_extra={'desc': 'a piece of text which may contains chemical reactions', '__dspy_field_type': 'input', 'prefix': 'Text:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    reactions = Field(annotation=Union[list[Reaction], NoneType] required=True json_schema_extra={'desc': 'the reactions extracted from 

In [11]:
class ClassifyReaction(dspy.Signature):
    """Giving the availability of extracting solid-state reaction formula from the text """
    text = dspy.InputField(desc='a piece of text which may contains solid-state chemical reaction formula')
    solid_state_definition = dspy.InputField(desc='the definition of solid-state reaction')
    
    extraction_availability_solid_react: int = dspy.OutputField(desc='1 for yes, 0 for no')


class Classify_CoT(dspy.Module):
    def __init__(self):
        self.predictor = dspy.ChainOfThought(signature=ClassifyReaction)

    def forward(self, text, solid_state_definition):
        prediction = self.predictor(text=text, solid_state_definition=solid_state_definition)
        return prediction
    

classifier = Classify_CoT()
definition = 'Solid-state reaction refers to a conventional method used in chemistry to synthesize various materials like ceramics and crystals by heating a mixture of raw materials in solid form.'

In [12]:
import re
from pydantic import ValidationError
from tenacity import retry, retry_if_exception_type
from sisyphus.utils.tenacity_retry_utils import pydantic_validate_retry_wraps
from sisyphus.utils.helper_functions import return_valid


exp_section_pattern = re.compile(r'\b(?:experiment(?:al|s|ing|ed)?|synthesis(?:es|ing|ed)?|preparation(?:s|al|ed|ing)?|process(?:es|ion|ing)?|method(?:s)?)\b', re.I)
def filter_with_kw(doc):
    return bool(exp_section_pattern.search(doc.metadata['sub_titles']))

@return_valid
@pydantic_validate_retry_wraps
def extract(doc, extract_program):
    # if classifier(text=doc.page_content, solid_state_definition=definition).extraction_availability_solid_react:
    text = doc.page_content
    title = doc.metadata['title']
    # abstract = doc.metadata['abstract']
    sub_titles = doc.metadata['sub_titles']
    # context = f'title:\n{title}\n' + f'abstract:\n{abstract}\n' + 'subtitles:\n' + '\n'.join([f'{sub_title}' for sub_title in sub_titles.split('/')]) + f'\nparagraph:\n{text}'
    context = f'title:\n{title}\n' + 'subtitles:\n' + '\n'.join([f'{sub_title}' for sub_title in sub_titles.split('/')]) + f'\nparagraph:\n{text}'
    prediction = extract_program(text=context)
    reactions = prediction.reactions
    return reactions

In [13]:
ARTICLE = '40_with_good_title'
TARGET = 'dspy_with_element_var_no_abstract_few_shot_no_lm_filter'
TARGET_ = 'dspy_with_element_var_no_abstract_zero_shot_no_lm_filter'
TARGET_vanilla = 'dspy_vanilla_no_abstract_zero_shot_no_lm_filter'
TARGET_vanilla_ = 'dspy_vanilla_no_abstract_zero_shot_optimized_no_lm_filter'

In [14]:
from functools import partial
from sisyphus.chain import Filter, Writer
from sisyphus.chain.customized_elements import customized_extractor
from sisyphus.utils.helper_functions import get_plain_articledb, get_create_resultdb
from sisyphus.chain.chain_elements import run_chains_with_extarction_history_multi_threads

article_db = get_plain_articledb(ARTICLE)
article_getter = Filter(article_db, filter_func=filter_with_kw, with_abstract=True)
result_db = get_create_resultdb(TARGET, Reaction)
result_db_u = get_create_resultdb(TARGET_, Reaction)
compiled_extractor = partial(extract, extract_program=compiled_program)
uncompiled_extractor = partial(extract, extract_program=uncompiled_program)
compiled_extract_el = customized_extractor(compiled_extractor, 'thread', 5)
uncompiled_extract_el = customized_extractor(uncompiled_extractor, 'thread', 5)

result_db_v = get_create_resultdb(TARGET_vanilla, Reaction)
vanilla_extractor = partial(extract, extract_program=vanilla_program)
vanilla_extract_el = customized_extractor(vanilla_extractor, 'thread', 5)
vanilla_chain = article_getter + vanilla_extract_el + Writer(result_db=result_db_v)

result_db_v_o = get_create_resultdb(TARGET_vanilla_, Reaction)
vanilla_extractor_o = partial(extract, extract_program=vanilla_program)
vanilla_extract_el_o = customized_extractor(vanilla_extractor_o, 'thread', 5)
vanilla_chain_o = article_getter + vanilla_extract_el_o + Writer(result_db=result_db_v_o)

compiled_chain = article_getter + compiled_extract_el + Writer(result_db=result_db)
uncompiled_chain = article_getter + uncompiled_extract_el + Writer(result_db=result_db_u)

e:\anaconda\envs\sisyphus\lib\site-packages\pydantic\main.py:1552: RuntimeWarning: fields may not start with an underscore, ignoring "__tablename__"
  warnings.warn(f'fields may not start with an underscore, ignoring "{f_name}"', RuntimeWarning)


In [15]:
from sisyphus.chain import Chain
compiled_extract_chain = article_getter + compiled_extract_el
uncompiled_extract_chain = article_getter + uncompiled_extract_el

save_chain = Chain(Writer(result_db=result_db))
save_chain_u = Chain(Writer(result_db=result_db_u))

In [16]:
CHAR_TO_HTML_LBS = {
    '/': '&sol;',
    '\\': '&bs;',
    '?': '&qm;',
    '*': '&st;',
    ':': '&cl;',
    '|': '&vb;',
    '<': '&lt;',
    '>': '&gt;',
    '"': '&quot;',
    '\'': '&apos;'
}
# convert the doi to escaped doi, and add .html to the end
def doi_to_escaped_doi(doi):
    escaped_doi = doi
    for char, html in CHAR_TO_HTML_LBS.items():
        escaped_doi = escaped_doi.replace(char, html)
    return escaped_doi + '.html'

def read_docinfo(docinfos):
    results = []
    for docinfo in docinfos:
        results.extend(docinfo.info)
    return results

In [10]:
bad_behaviour_dois = [
    "10.1002/adfm.201101123",
    "10.1039/c1cy00199j",
    "10.1016/j.jpcs.2005.02.003"
]

In [13]:
docinfo = uncompiled_extract_chain.compose(doi_to_escaped_doi(bad_behaviour_dois[2]))
docinfo_ = compiled_extract_chain.compose(doi_to_escaped_doi(bad_behaviour_dois[2]))

In [14]:
read_docinfo(docinfo)

[Reaction(precursors=['BaS', 'Ga2S3'], additives=['EuF3', 'H2S', 'Ar'], target=Target(target_formula='BaGa2S4:Eu2+', amount_var={}, element_var={}), reaction_type='others'),
 Reaction(precursors=['BaCO3', 'Ga2O3', 'Eu2O3'], additives=['H2S'], target=Target(target_formula='BaGa2S4:Eu2+', amount_var={}, element_var={}), reaction_type='others'),
 Reaction(precursors=['Ba', 'Ga', 'EuS'], additives=[], target=Target(target_formula='BaGa2S4:Eu2+', amount_var={}, element_var={}), reaction_type='solid-state')]

In [15]:
read_docinfo(docinfo_)

[Reaction(precursors=['BaS', 'Ga2S3', 'EuF3'], additives=[], target=Target(target_formula='BaGa2S4:Eu2+', amount_var={'Eu2+': [0.01, 0.05, 0.1]}, element_var={}), reaction_type='solid-state')]

In [21]:
lm.inspect_history(5)





[2024-12-12T20:14:27.877201]

System message:

Your input fields are:
1. `text` (str): a piece of text which may contains solid-state chemical reaction formula
2. `solid_state_definition` (str): the definition of solid-state reaction

Your output fields are:
1. `reasoning` (str)
2. `extraction_availability_solid_react` (int): 1 for yes, 0 for no

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## solid_state_definition ## ]]
{solid_state_definition}

[[ ## reasoning ## ]]
{reasoning}

[[ ## extraction_availability_solid_react ## ]]
{extraction_availability_solid_react}        # note: the value you produce must be a single int value

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Giving the availability of extracting solid-state reaction formula from the text


User message:

[[ ## text ## ]]
Synthesis of Mesoporous LTO: Poly(isoprene) -b-poly(ethylene oxide) block copoly

In [15]:
run_chains_with_extarction_history_multi_threads(compiled_chain, 'articles_processed', 10, 'few-shot-no-lm-filter')

100%|██████████| 40/40 [00:43<00:00,  1.08s/it]


In [16]:
run_chains_with_extarction_history_multi_threads(uncompiled_chain, 'articles_processed', 10, 'zero-shot-no-lm-filter')

100%|██████████| 40/40 [00:44<00:00,  1.12s/it]


In [11]:
run_chains_with_extarction_history_multi_threads(vanilla_chain, 'articles_processed', 10, 'vanilla-zero-shot-no-lm-filter')

 57%|█████▊    | 23/40 [00:21<00:06,  2.51it/s]ERROR:root:1 validation error for nullable[list[Reaction]]
0.reaction_type
  Input should be 'solid-state', 'sol-gel', 'co-precipitation', 'hydrothermal', 'flux' or 'others' [type=literal_error, input_value='high-pressure', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
NoneType: None
 70%|███████   | 28/40 [00:24<00:06,  1.97it/s]ERROR:root:2 validation errors for nullable[list[Reaction]]
0.reaction_type
  Input should be 'solid-state', 'sol-gel', 'co-precipitation', 'hydrothermal', 'flux' or 'others' [type=literal_error, input_value='reduction', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error
1.reaction_type
  Input should be 'solid-state', 'sol-gel', 'co-precipitation', 'hydrothermal', 'flux' or 'others' [type=literal_error, input_value='steam reforming', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v

In [17]:
run_chains_with_extarction_history_multi_threads(vanilla_chain_o, 'articles_processed', 10, 'vanilla-zero-shot-optimized-no-lm-filter')

100%|██████████| 40/40 [00:44<00:00,  1.12s/it]


In [17]:
results = result_db.load_as_json(with_doi=True)
import json
with open('dspy_with_element_var_no_abstract_few_shot_no_lm_filter.json', 'w') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [18]:
results = result_db_u.load_as_json(with_doi=True)
import json
with open('dspy_with_element_var_no_abstract_zero_shot_no_lm_filter.json', 'w') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [12]:
results = result_db_v.load_as_json(with_doi=True)
import json
with open('dspy_vanilla_no_abstract_no_llm_filter.json', 'w') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [18]:
results = result_db_v_o.load_as_json(with_doi=True)
import json
with open('dspy_vanilla_no_abstract_optimized_no_llm_filter.json', 'w') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [12]:
text = "Fe2O3 and Y2O3 were used as starting materials to prepare YFeO3 by hydrothermal method, sol–gel and solid state reaction in most previous work,23–25 which requires the calcination step at a high temperature of above 700 °C. The reaction is heterogeneous due to the poor solubility of the used precursors in the reaction medium. In our present work, the use of water soluble Fe(NO3)3 and Y(NO3)3 precursors enables the adequate mixing and thus the homogenous reaction, benefiting the formation of monophasic YFeO3 in our synthetic strategy although it is thermodynamically unstable."

In [13]:
compiled_program(text=text).reactions

[Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='hydrothermal'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='sol-gel'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='solid-state'),
 Reaction(precursors=['Fe(NO3)3', 'Y(NO3)3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='others')]

In [19]:
lm.inspect_history(5)





[2024-12-13T22:33:40.862194]

System message:

Your input fields are:
1. `text` (str): a piece of text which may contains chemical reactions

Your output fields are:
1. `reactions` (Union[list[Reaction], NoneType]): the reactions extracted from the text, return null if no reaction found

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reactions ## ]]
{reactions}        # note: the value you produce must be pareseable according to the following JSON schema: {"$defs": {"Reaction": {"type": "object", "properties": {"additives": {"type": "array", "items": {"type": "string"}, "title": "Additives"}, "precursors": {"type": "array", "description": "ensure it is a valid chemical formula", "items": {"type": "string"}, "title": "Precursors"}, "reaction_type": {"type": "string", "enum": ["solid-state", "sol-gel", "co-precipitation", "hydrothermal", "flux", "others"], "title": "Reaction Type"}, "target": {"$ref": "

In [15]:
uncompiled_program(text=text).reactions

[Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='hydrothermal'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='sol-gel'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='solid-state'),
 Reaction(precursors=['Fe(NO3)3', 'Y(NO3)3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='others')]

In [16]:
lm.inspect_history()





[2024-12-13T11:44:13.061874]

System message:

Your input fields are:
1. `text` (str): a piece of text which may contains chemical reactions

Your output fields are:
1. `reasoning` (str)
2. `reactions` (Union[list[Reaction], NoneType]): the reactions extracted from the text, return null if no reaction found

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## reactions ## ]]
{reactions}        # note: the value you produce must be pareseable according to the following JSON schema: {"$defs": {"Reaction": {"type": "object", "properties": {"additives": {"type": "array", "items": {"type": "string"}, "title": "Additives"}, "precursors": {"type": "array", "description": "ensure it is a valid chemical formula", "items": {"type": "string"}, "title": "Precursors"}, "reaction_type": {"type": "string", "enum": ["solid-state", "sol-gel", "co-precipitation", "hydrothermal", "flux", "o

In [14]:
lm.history[-2]

{'prompt': None,
 'messages': [{'role': 'user',
   'content': 'This is an example of the task, though some input or output fields are not supplied.\n\n[[ ## text ## ]]\nThe BaCe0.7In0.15Ta0.05Y0.1O3-δ (BCITY) powder was prepared by combing solid state reaction and Pechini method to ensure homogeneity of mixed oxides. Firstly, the Ba(NO3)2, Ce(NO3)3⋅6H2O, In(NO3)3⋅4.5H2O, and Y(NO3)3⋅6H2O with the stoichiometric amounts were dissolved in deionized water. Secondly, the citric acid, as a complexation agent, was added with the molar ratio (3:2) of citric acid to metal ions. After the pH value of the solution was adjusted to be 7 by adding appropriate ammonia water, the stoichiometric amount of Ta2O5 was added. Thirdly, the solution added with Ta2O5 was stirred continuously during heating to evaporate water until it was changed into viscous gel and finally ignited to flame, which resulted in the formation of some white ashes. Finally, the BCITY powder was obtained by calcining the white ash

In [17]:
compiled_program(text=text).reactions

[Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='hydrothermal'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='sol-gel'),
 Reaction(precursors=['Fe2O3', 'Y2O3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='solid-state'),
 Reaction(precursors=['Fe(NO3)3', 'Y(NO3)3'], additives=[], target=Target(target_formula='YFeO3', amount_var={}, element_var={}), reaction_type='others')]

In [45]:
lm.inspect_history()





[2024-12-12T15:42:34.849318]

User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## text ## ]]
The BaCe0.7In0.15Ta0.05Y0.1O3-δ (BCITY) powder was prepared by combing solid state reaction and Pechini method to ensure homogeneity of mixed oxides. Firstly, the Ba(NO3)2, Ce(NO3)3⋅6H2O, In(NO3)3⋅4.5H2O, and Y(NO3)3⋅6H2O with the stoichiometric amounts were dissolved in deionized water. Secondly, the citric acid, as a complexation agent, was added with the molar ratio (3:2) of citric acid to metal ions. After the pH value of the solution was adjusted to be 7 by adding appropriate ammonia water, the stoichiometric amount of Ta2O5 was added. Thirdly, the solution added with Ta2O5 was stirred continuously during heating to evaporate water until it was changed into viscous gel and finally ignited to flame, which resulted in the formation of some white ashes. Finally, the BCITY powder was obtained by calcining the white ashes at 1000 °C for

In [46]:
compiled_program.predictor.extended_signature

StringSignature(text -> reasoning, reactions
    instructions='Your objective is to extract all chemical reactions described in the text, regardless of prominence. Each reaction should include:\n1. Explicitly mentioned precursors and target products.\n2. Reaction type, as classified into solid-state, sol-gel, hydrothermal, co-precipitation, flux, or others.\n\nEnsure reasoning includes all reactions described, mentioning each reaction process briefly.\n\nExamples are provided for reference.'
    text = Field(annotation=str required=True json_schema_extra={'desc': 'a piece of text which may contains chemical reactions', '__dspy_field_type': 'input', 'prefix': 'Text:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    reactions = Field(annotation=Union[list[Reaction], NoneType] required=True json_schema_extra={'desc': 'the reactions extracted from 

In [ ]:
dspy.COPRO
dspy.ChainOfThought